In [5]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

def save_mel_spectrogram(audio_path, save_path):
    y, sr = librosa.load(audio_path, sr=None)
    mel_spect = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    mel_spect_db = librosa.power_to_db(mel_spect, ref=np.max)
    
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spect_db, sr=sr, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel spectrogram')
    plt.tight_layout()
    plt.axis('off')
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    plt.close()

kiwi_dir = "C:\\Users\\44778\\OneDrive\\Desktop\\UWE_Docs\\7. Dissertation\\Xeno Canto Audio Files\\Kiwi Calls"
other_dir = "C:\\Users\\44778\\OneDrive\\Desktop\\UWE_Docs\\7. Dissertation\\Xeno Canto Audio Files\\Other Calls"
kiwi_save_dir = "C:\\Users\\44778\\OneDrive\\Desktop\\UWE_Docs\\7. Dissertation\\Xeno Canto Audio Files\\Kiwi Spectograms"
other_save_dir = "C:\\Users\\44778\\OneDrive\\Desktop\\UWE_Docs\\7. Dissertation\\Xeno Canto Audio Files\\Other Spectograms"

os.makedirs(kiwi_save_dir, exist_ok=True)
os.makedirs(other_save_dir, exist_ok=True)

for filename in os.listdir(kiwi_dir):
    audio_path = os.path.join(kiwi_dir, filename)
    save_path = os.path.join(kiwi_save_dir, filename.replace('.mp3', '.png'))
    save_mel_spectrogram(audio_path, save_path)

for filename in os.listdir(other_dir):
    audio_path = os.path.join(other_dir, filename)
    save_path = os.path.join(other_save_dir, filename.replace('.mp3', '.png'))
    save_mel_spectrogram(audio_path, save_path)


In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class BirdDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = [os.path.join(image_dir, file) for file in os.listdir(image_dir) if file.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert('RGB')
        label = 0 if 'kiwi' in img_path else 1
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

kiwi_dataset = BirdDataset(kiwi_save_dir, transform=transform)
other_dataset = BirdDataset(other_save_dir, transform=transform)

full_dataset = kiwi_dataset + other_dataset
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN()


In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Accuracy: {100 * correct / total}%")


Epoch 1, Loss: 0.23040019227191522
Validation Accuracy: 100.0%
Epoch 2, Loss: 0.0
Validation Accuracy: 100.0%
Epoch 3, Loss: 0.0
Validation Accuracy: 100.0%
Epoch 4, Loss: 0.0
Validation Accuracy: 100.0%
Epoch 5, Loss: 0.0
Validation Accuracy: 100.0%
Epoch 6, Loss: 0.0
Validation Accuracy: 100.0%
Epoch 7, Loss: 0.0
Validation Accuracy: 100.0%
Epoch 8, Loss: 0.0
Validation Accuracy: 100.0%
Epoch 9, Loss: 0.0
Validation Accuracy: 100.0%
Epoch 10, Loss: 0.0
Validation Accuracy: 100.0%


In [12]:
torch.save(model.state_dict(), 'kiwi_bird_classifier.pth')